In [1]:
import os
import cv2
import PIL.Image as Image
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from enhance_model import enhance_net

In [2]:
import tensorflow as tf
tf.version.VERSION

'2.10.1'

In [3]:
def NormalizeData(data):
    return (data - tf.math.reduce_min(data)) / (tf.math.reduce_max(data) - tf.math.reduce_min(data))

In [4]:
def image_selection(enhancement_images):
    images = []
    select_img = []
    select_img_safe = [[],[],[],[],[],[],[],[]]
    for index, i in enumerate(enhancement_images):
        r,g,b = tf.split(i, 3, axis=2)
        r_g = int(np.abs(np.mean(r*0.299-g*0.587)))
        r_b = int(np.abs(np.mean(r*0.299-b*0.114)))
        g_b = int(np.abs(np.mean(g*0.587-b*0.114)))
        
        img = tf.image.rgb_to_hsv(i)
        #h存儲圖像的色調通道，s存儲圖像的飽和度通道，v存儲圖像的亮度通道
        h,s,v = tf.split(img, 3, axis=2)
        mean_hue = tf.math.reduce_mean(h*255)
        mean_v = tf.math.reduce_mean(v)
        

        if 22 < mean_hue and mean_hue <= 100:
            select_img_safe[index].append('hue')
            if 30 <= mean_v and mean_v <= 145:
                select_img_safe[index].append('lightness')
                if  5<=r_g and  r_g <= 36: 
                    select_img_safe[index].append('r-g')
                    if 5 <= r_b and r_b<=48:
                        select_img_safe[index].append('r-b')
                        if 5 <= g_b and  g_b <= 75:
                            select_img_safe[index].append('g-b')
                            images.append(i)
                            select_img.append(index)
    if np.shape(images)[0] == 0:
        print(select_img_safe)
        return enhancement_images[0:5]
    print(select_img)
    return images

In [5]:
# model_name = ['DCE', 'CSP_DCE', 'MSP_DCE','DCE++', 'Dil_DCE']
model_name = ['Dil_DCE']
model_name = model_name[0]

print(model_name)

Dil_DCE


In [6]:
i = 10
path = './model/{0}/weights/epoch{1}/'.format(model_name, i)
# path = './model/DCE_1_10_5_1600/weights/epoch{0}/'.format(i)
model = enhance_net(input_shape=(None,None,3), model_name=model_name)
model.enhancement_net.load_weights(path)    

c:\Users\user\anaconda3\envs\TF2\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [7]:
# img_dir = './Dataset/denoise/LOL/test/low/'
# # img_dir = './result/paper_use/SICE-Visual Comparisons/low/'
# # img_dir = 'C://Users/user/Downloads/mh/'
# output_dir = './result/{0}/epoch{1}/'.format(model_name, i)
    
# folder = os.path.exists(output_dir)
# if not folder:
#     os.makedirs(output_dir)
# for i in range(8):
#     strat = time.time()
#     for file in os.listdir(img_dir):
#         img_path = img_dir + file
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
        
#         h,w,c = np.shape(img)
#         img = tf.reshape(img, (1,h,w,c))    
#         img = tf.cast(img, dtype=tf.float32)
        
#         enhancemnet_image, parameter_map = model.predict(img)
        
#         enhancemnet_image = NormalizeData(enhancemnet_image)
#         enhancemnet_image = np.reshape(enhancemnet_image, (h,w,c))
#         enhancemnet_image = cv2.cvtColor(enhancemnet_image, cv2.COLOR_RGB2BGR)
#         # cv2.imwrite(output_dir +file, enhancemnet_image*255)

#     end = time.time()
#     per_image_time = (end - strat)/len(os.listdir(img_dir))
#     print(per_image_time)
        

In [8]:
# i = 2
for i in range(10,101,10):
    path = './model/{0}/weights/epoch{1}/'.format(model_name, i)
    # path = './model/DCE_1_10_5_1600/weights/epoch{0}/'.format(i)
    model = enhance_net(input_shape=(None,None,3), model_name=model_name)
    model.enhancement_net.load_weights(path)    
    
    # img_dir = './Dataset/denoise/LOL/train/low/'
    # img_dir = './Dataset/denoise/test/low/'
    # img_dir = 'C://Users/user/Downloads/mh/'
    # img_dir = './Dataset/low_LOL_485/'
    # img_dir = './Dataset/low_LOL_15/'
    # img_dir = './Dataset/low_LSRW_5600/'
    # img_dir = './Dataset/low_LSRW_50/'
    img_dir = './Dataset/low/'

    output_dir = './result/{0}/epoch{1}/'.format(model_name, i)

    folder = os.path.exists(output_dir)
    if not folder:
        os.makedirs(output_dir)
    pad_weight = False
    pad_height = False
    strat = time.time()
    for file in os.listdir(img_dir):
        img_path = img_dir + file
        img = cv2.imread(img_path)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
        h,w,c = np.shape(img)
        img = tf.reshape(img, (1,h,w,c))
        pred_img = tf.cast(img, dtype=tf.float32)
        if (pred_img.shape[2] %2 ) !=   0:
            padding = tf.constant([[0, 0], [0, 0], [0, 1], [0, 0]])
            pred_img = tf.pad(pred_img, padding, mode='SYMMETRIC')
            pad_weight=True
        if (pred_img.shape[1] % 2 ) != 0:
            padding = tf.constant([[0, 0], [0, 1], [0, 0], [0, 0]])
            pred_img = tf.pad(pred_img, padding, mode='SYMMETRIC')
            pad_height=True
        enhancemnet_image, parameter_map = model.predict(pred_img)
        if  pad_weight == True:
            enhancemnet_image = enhancemnet_image[:, :, :-1]
            pad_weight=False
        if  pad_height == True:
            enhancemnet_image = enhancemnet_image[:, :-1, :]
            pad_height=False

        enhancemnet_image = NormalizeData(enhancemnet_image)
        enhancemnet_image = np.reshape(enhancemnet_image, (h,w,c))
        enhancemnet_image = cv2.cvtColor(enhancemnet_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_dir +file, enhancemnet_image*255)
    
    
        
    end = time.time()
    
    per_image_time = (end - strat)/len(os.listdir(img_dir))
    print(f"Epoch {i} took {end - strat} seconds to process {len(os.listdir(img_dir))} images. Average time per image: {per_image_time} seconds.")

1/1 [==============================] - 0s 16ms/step
Epoch 10 took 6.912559270858765 seconds to process 25 images. Average time per image: 0.2765023708343506 seconds.
1/1 [==============================] - 0s 16ms/step
Epoch 20 took 4.012214660644531 seconds to process 25 images. Average time per image: 0.16048858642578126 seconds.
1/1 [==============================] - 0s 16ms/step
Epoch 30 took 4.031976222991943 seconds to process 25 images. Average time per image: 0.16127904891967773 seconds.
1/1 [==============================] - 0s 15ms/step
Epoch 40 took 3.725029230117798 seconds to process 25 images. Average time per image: 0.1490011692047119 seconds.
1/1 [==============================] - 0s 17ms/step
Epoch 50 took 3.919907808303833 seconds to process 25 images. Average time per image: 0.15679631233215333 seconds.
1/1 [==============================] - 0s 16ms/step
Epoch 60 took 3.724407911300659 seconds to process 25 images. Average time per image: 0.14897631645202636 seconds.


In [9]:
# import os
# import cv2
# import PIL.Image as Image
# import time
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from enhanced_model import enhance_net

# def NormalizeData(data):
#     return (data - np.min(data)) / (np.max(data) - np.min(data))

# model_name = ['DCE', 'CSP_DCE','MSP_DCE', 'DCE++']
# model_name = model_name[1]

# print(model_name)

# i = 10
# path = './model/{0}/weights/epoch{1}/'.format('CSP_DCE', i)
# model = enhance_net(input_shape=(None,None,3), model_name=model_name)
# model.enhancement_net.load_weights(path) 

# # img_path = 'parameter_map_median.55_1.png'
# img_path = './result/paper_use/SICE-Visual Comparisons/low/17-1.JPG'
# img = cv2.imread(img_path)

# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
# h,w,c = np.shape(img)
# img = tf.reshape(img, (1,h,w,c))
# pred_img = tf.cast(img, dtype=tf.float32)

# enhancement_image, parameter_map = model.predict(pred_img)

In [10]:
# enhancement_image = np.reshape(enhancement_image, (h,w,c))
# enhancement_image = cv2.cvtColor(enhancement_image, cv2.COLOR_RGB2BGR)
# enhancement_image = NormalizeData(enhancement_image)
# cv2.imwrite('enhanced_img.png', enhancement_image*255)

In [11]:
# parameter_map = np.reshape(parameter_map, (h,w,c))
# parameter_map = cv2.cvtColor(parameter_map, cv2.COLOR_RGB2BGR)
# p_r, p_g, p_b = cv2.split(parameter_map)
# p_r = NormalizeData(p_r)
# p_g = NormalizeData(p_g)
# p_b = NormalizeData(p_b)
# cv2.imwrite('parameter_map_R.png', p_r*255)
# cv2.imwrite('parameter_map_G.png', p_g*255)
# cv2.imwrite('parameter_map_B.png', p_b*255)

In [12]:
# plt.figure(figsize=(15, 15))

# plt.subplot(1, 3, 1)
# plt.imshow(p_r, cmap='gray')
# plt.subplot(1, 3, 2)
# plt.imshow(p_g, cmap='gray')
# plt.subplot(1, 3, 3)
# plt.imshow(p_b, cmap='gray')
# plt.show()

In [13]:
# #分割成2個陣列的parameter_maps
# A13,A24 = np.split(parameter_map,2,axis=0)

In [14]:
# A1,A3 = np.split(A13,2,axis=1)
# A2,A4 = np.split(A24,2,axis=1)

# cv2.imwrite('parameter_map.png', parameter_map*255)
# cv2.imwrite('A1.png', A1*255)
# cv2.imwrite('A2.png', A2*255)
# cv2.imwrite('A3.png', A3*255)
# cv2.imwrite('A4.png', A4*255)

In [15]:
# plt.figure(figsize=(15, 15))

# plt.subplot(1, 4, 1)
# plt.imshow(A1, cmap='gray')
# plt.subplot(1, 4, 2)
# plt.imshow(A2, cmap='gray')
# plt.subplot(1, 4, 3)
# plt.imshow(A3, cmap='gray')
# plt.subplot(1, 4, 4)
# plt.imshow(A4, cmap='gray')
# plt.show()

In [16]:
# plt.figure(figsize=(15, 15))

# plt.subplot(1, 3, 1)
# plt.imshow(p_r, cmap='gray')
# plt.subplot(1, 3, 2)
# plt.imshow(p_g, cmap='gray')
# plt.subplot(1, 3, 3)
# plt.imshow(p_b, cmap='gray')
# plt.show()

In [17]:
# #heatmap 
# plt.figure(figsize=(15, 15))

# plt.subplot(1, 3, 1)
# plt.imshow(p_r, cmap='hot')
# plt.subplot(1, 3, 2)
# plt.imshow(p_g, cmap='hot')
# plt.subplot(1, 3, 3)
# plt.imshow(p_b, cmap='hot')
# plt.show()

In [18]:
# A1,A3 = np.split(A13,2,axis=1)
# A2,A4 = np.split(A24,2,axis=1)

# cv2.imwrite('parameter_map.png', parameter_map*255)
# cv2.imwrite('A1.png', A1*255)
# cv2.imwrite('A2.png', A2*255)
# cv2.imwrite('A3.png', A3*255)
# cv2.imwrite('A4.png', A4*255)

In [19]:
# plt.figure(figsize=(15, 15))

# plt.subplot(1, 4, 1)
# plt.imshow(A1, cmap='gray')
# plt.subplot(1, 4, 2)
# plt.imshow(A2, cmap='gray')
# plt.subplot(1, 4, 3)
# plt.imshow(A3, cmap='gray')
# plt.subplot(1, 4, 4)
# plt.imshow(A4, cmap='gray')
# plt.show()

In [20]:
# import cv2
# import matplotlib.pyplot as plt

# # 讀取R、G、B參數圖
# parameter_map_R = cv2.imread('parameter_map_R.png', cv2.IMREAD_GRAYSCALE)
# parameter_map_G = cv2.imread('parameter_map_G.png', cv2.IMREAD_GRAYSCALE)
# parameter_map_B = cv2.imread('parameter_map_B.png', cv2.IMREAD_GRAYSCALE)

# # 正規化參數圖的值範圍到 [0, 1]
# normalized_map_R = parameter_map_R / 255.0
# normalized_map_G = parameter_map_G / 255.0
# normalized_map_B = parameter_map_B / 255.0

# # 顯示熱力圖
# plt.figure(figsize=(15, 5))
# #hot#jet
# plt.subplot(1, 3, 1)
# plt.imshow(normalized_map_R, cmap='jet')
# plt.title('Heatmap R')
# plt.axis('off')
# #hot#jet
# plt.subplot(1, 3, 2)
# plt.imshow(normalized_map_G, cmap='jet')
# plt.title('Heatmap G')
# plt.axis('off')

# plt.subplot(1, 3, 3)
# #hot#jet
# plt.imshow(normalized_map_B, cmap='jet')
# plt.title('Heatmap B')
# plt.axis('off')

# plt.tight_layout()
# plt.show()

In [21]:
# import cv2
# import matplotlib.pyplot as plt

# image = cv2.imread('enhanced_img.png')


# plt.figure(figsize=(10, 10))
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [22]:
# import cv2
# import matplotlib.pyplot as plt

# image = cv2.imread('enhanced_img.png')

# b, g, r = cv2.split(image)  # 分離 R、G、B 通道
# b_median = cv2.medianBlur(b, 3)  # 對藍色通道應用中值濾波器
# g_median = cv2.medianBlur(g, 3)  # 對綠色通道應用中值濾波器
# r_median = cv2.medianBlur(r, 3)  # 對紅色通道應用中值濾波器

# # 合併處理後的通道
# processed_image = cv2.merge([b_median, g_median, r_median])

# plt.figure(figsize=(10, 10))
# plt.imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()
# cv2.imwrite('enhanced_img_median.55_2.png', processed_image)

In [23]:
# import cv2
# import matplotlib.pyplot as plt

# # 讀取合併的參數圖像
# parameter_map_RGB = cv2.imread('parameter_map.png', cv2.IMREAD_COLOR)

# # 分離R、G、B參數圖
# parameter_map_R = parameter_map_RGB[:, :, 2]  # 紅色通道
# parameter_map_G = parameter_map_RGB[:, :, 1]  # 綠色通道
# parameter_map_B = parameter_map_RGB[:, :, 0]  # 藍色通道

# # 正規化參數圖像的值範圍為 [0, 1]
# normalized_map_R = parameter_map_R / 255.0
# normalized_map_G = parameter_map_G / 255.0
# normalized_map_B = parameter_map_B / 255.0

# # 顯示參數圖像
# plt.figure(figsize=(15, 5))

# plt.subplot(1, 3, 1)
# plt.imshow(normalized_map_R, cmap='hot')
# plt.title('Parameter Map R')
# plt.axis('off')

# plt.subplot(1, 3, 2)
# plt.imshow(normalized_map_G, cmap='hot')
# plt.title('Parameter Map G')
# plt.axis('off')

# plt.subplot(1, 3, 3)
# plt.imshow(normalized_map_B, cmap='hot')
# plt.title('Parameter Map B')
# plt.axis('off')

# plt.tight_layout()
# plt.show()
